In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import utils

In [15]:
data = utils.load_data('../data/output.csv')

In [16]:
data.sample(5)

,Evento,Puerta,FechaHora,Rol
32803,Acceso concedido,ML-VEHICULAR-ML-T11,2024/10/01 20:52:07,ESTUDIANTE
31311,Acceso concedido,ML-VEHICULAR-ML-T5,2024/10/01 05:23:59,ESTUDIANTE
6225,Acceso concedido,CD7-MOL4-IN-T1,2024/10/01 12:53:28,ASEO
6056,Acceso concedido,ML-VEHICULAR-ML-T2,2024/10/01 05:20:17,PROFESOR
58144,Acceso concedido,CD3-MOL6-IN-T7,2024/10/01 07:17:35,EMPLEADO


In [17]:
data['custom_index'] = (data['Rol']).apply(utils.create_index)
data = data.sort_values(by='FechaHora').reset_index().drop(columns='index')
data.head(100000)

,Evento,Puerta,FechaHora,Rol,custom_index
0,Acceso concedido,SD-VEHICULAR-SD-T5,2024/10/01 00:00:05,ASEO,16857
1,Acceso concedido,ML7-MOL3-IN-T5,2024/10/01 00:00:09,VIGILANCIA,19087
2,Acceso concedido,CD9-MOL4-OUT-T7,2024/10/01 00:00:25,VIGILANCIA,19625
3,Acceso concedido,RGD6-MOL1-OUT-T11,2024/10/01 00:00:25,ASEO,16496
4,Acceso concedido,ML-VEHICULAR-ML-T7,2024/10/01 00:00:34,ASEO,16283
...,...,...,...,...,...
99995,Acceso concedido,ML-VEHICULAR-ML-T10,2024/10/01 23:59:54,PROFESOR,4061
99996,Acceso concedido,CD9-MOL4-OUT-T10,2024/10/01 23:59:54,PROFESOR,4705
99997,Acceso concedido,SD11-MOL1-OUT-T7,2024/10/01 23:59:56,EMPLEADO,18110
99998,Acceso concedido,SD-VEHICULAR-SD-T6,2024/10/01 23:59:57,EMPLEADO,18410


In [18]:
import cv2
import numpy as np
import pytesseract

# Configurar la ruta de Tesseract-OCR (asegúrate de que esté instalado)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # For Windows

# Cargar las imágenes
images = [
    cv2.imread('../assets/Piso1.png'),
    cv2.imread('../assets/Piso2.png'),
    cv2.imread('../assets/Piso5.png'),
    cv2.imread('../assets/Piso6.png'),
]

for i, image in enumerate(images):

    # Convertir la imagen a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir los límites del color rojo en el espacio HSV
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear máscaras para ambos rangos de rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Unir las dos máscaras para cubrir todas las tonalidades de rojo
    mask = mask1 + mask2

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una lista para almacenar las coordenadas de los cuadrados
    square_coords = []

    # Filtrar solo los contornos que sean cuadrados y que tengan un área significativa
    for contour in contours:
        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar si el contorno tiene 4 lados (es un cuadrado o rectángulo)
        if len(approx) == 4 and cv2.contourArea(contour) > 100:  # Filtrar por área
            square_coords.append(approx)

            # Extraer la región del cuadrado detectado de la imagen original
            x, y, w, h = cv2.boundingRect(approx)
            square_roi = image[y:y + h, x:x + w]

            # Preprocesar la imagen antes de OCR (convertir a gris y aplicar umbral)
            gray_square = cv2.cvtColor(square_roi, cv2.COLOR_BGR2GRAY)

            # Binarizar la imagen (solo si es necesario)
            _, threshold_square = cv2.threshold(gray_square, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Usar OCR para leer el texto dentro del cuadrado (configurado para una línea)
            text = pytesseract.image_to_string(threshold_square, config='--psm 7')  # psm 7 para una línea de texto

            # Decidir el color basado en el texto
            if "WAIRA" in text.upper():
                color = (0, 255, 0)  # Verde si el texto es "APPROVED"
            elif "ML-603" in text.upper():
                color = (0, 0, 255)  # Rojo si el texto es "REJECTED"
            else:
                color = (255, 255, 0)  # Amarillo para otros casos

            # Dibujar el cuadrado completo con el color basado en el texto
            cv2.fillPoly(image, [approx], color)

            # Poner el texto extraído en la imagen
            center_x = x + w // 2
            center_y = y + h // 2

            text_size = cv2.getTextSize(text.strip(), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_width, text_height = text_size
            text_x = center_x - text_width // 2
            text_y = center_y + text_height // 2
            text_color = (255, 255, 255)
            cv2.putText(image, text.strip(), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, text_color, 5, cv2.LINE_AA)

    # Guardar la imagen con los cuadrados detectados y coloreados
    cv2.imwrite(f'../assets/out/{i}.jpg', image)




    # # Imprimir las coordenadas de las esquinas de los cuadrados detectados
    # for i, square in enumerate(square_coords):
    #     print(f'Cuadrado {i+1}:')
    #     for point in square:
    #         x, y = point[0]
    #         print(f'    Esquina: (x: {x}, y: {y})')


## Procesado de datos Oferta de cursos

In [19]:
#%pip install polars
import polars as pl
from ast import literal_eval

In [20]:
df = pl.read_csv('../../scrap/formatted_data.csv', infer_schema_length=10000)

In [21]:
df.sample(5)

Course Code,Course Name,Cupo,NRC,Sección,Créditos,Periodo,Parte de Periodo,Campus,Schedule Table,Cupos Ofrecidos,Cupos Inscritos,Cupos Disponibles
str,str,str,i64,str,i64,i64,str,str,str,i64,i64,i64
"""CBCA-1433""","""NUEVO CINE LATINOAMERICANO: UN…","""7""",73286,"""02""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': '1800-19…",50,43,7
"""PATO-5303""","""QX GASTROINTESTINAL III""","""1""",70616,"""01""",4,202420,"""POSGRADO MEDICINA - 26 SEMANAS""","""EXTERNO UNIVERSIDAD""","""[{'Día': '-', 'Hora': '.', 'Sa…",4,3,1
"""DISO-3199""","""TRABAJO DE GRADO""","""Sin cupo""",63900,"""35""",4,202420,"""PERIODO NO RETIRABLE""","""CAMPUS PRINCIPAL""","""[{'Día': '-', 'Hora': '.', 'Sa…",3,3,0
"""MATE-1207""","""CÁLCULO VECTORIAL""","""10""",39061,"""E""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'I', 'Hora': 'V', 'Sa…",81,71,10
"""MEDI-3442""","""INTERNADO OBLIGATORIO URGENCIA…","""Sin cupo""",73236,"""05""",3,202420,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","""[{'Día': 'L', 'Hora': 'M', 'Sa…",4,4,0


In [22]:
# change the column type to list
df = df.with_columns(
    pl.col("Schedule Table").map_elements(literal_eval).alias("Schedule Table")
)

C:\Users\Leo\AppData\Local\Temp\ipykernel_37172\2190451217.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


In [23]:
df.sample(10)

Course Code,Course Name,Cupo,NRC,Sección,Créditos,Periodo,Parte de Periodo,Campus,Schedule Table,Cupos Ofrecidos,Cupos Inscritos,Cupos Disponibles
str,str,str,i64,str,i64,i64,str,str,list[struct[6]],i64,i64,i64
"""ICYA-4217""","""TESIS 1 MIAMB""","""48""",25843,"""01""",4,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",50,2,48
"""MUSI-1323""","""INSTRUMENTO PRINCIPAL 2 GUIT""","""Sin cupo""",70326,"""02""",4,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",1,1,0
"""IELE-4005""","""PROYECTO ESPECIAL I""","""No info""",11542,"""01""",4,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"",""."",""2024-08-05"",""2024-11-30"",""."",""No info""}]",0,10,-10
"""ESCR-1101""","""ESCRITURA UNIVERSITARIA I (CIC…","""Sin cupo""",72593,"""99""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""1400-1550"","".AU_107"",""2024-08-05"",""2024-09-28"","".Edif. Aulas""}, {""I"",""1400-1550"","".C_213"",""2024-08-05"",""2024-09-28"","".Bloque C""}]",24,24,0
"""ARQT-2601""","""PENSAMIENTO COMPUTACIONAL""","""Sin cupo""",52312,"""02""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""I"",""0800-1050"","".C_201"",""2024-08-05"",""2024-11-30"","".Bloque C""}]",20,23,-3
"""CBIO-6982""","""TESIS DOCTORADO 3""","""4""",12006,"""01""",8,202420,"""PERIODO NO RETIRABLE""","""CAMPUS PRINCIPAL""","[{""-"",""."",""2024-08-05"",""2024-11-30"",""."",""No info""}]",5,1,4
"""ICYA-4214""","""TESIS 2 MICIV""","""26""",25841,"""01""",8,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",50,24,26
"""EGOB-3604""","""ASUNTOS PÚBLICOS GLOBALES""","""12""",72724,"""01""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""0800-0920"","".R_113"",""2024-08-05"",""2024-11-30 .Edif. Richard (R)""}]",70,58,12
"""EMAT-4801""","""NOCIÓN DE CURRÍCULO EN MATEMÁT…","""17""",55140,"""01""",4,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""S"",""0800-1150"","".SD_715"",""2024-08-17"",""2024-08-18"","".Edif. J.M.Santodomingo (SD)""}, {""S"",""0800-1150"","".SD_715"",""2024-08-31"",""2024-09-01"","".Edif. J.M.Santodomingo (SD)""}, … {""S"",""0800-1150"","".SD_715"",""2024-09-28"",""2024-09-29"","".Edif. J.M.Santodomingo (SD)""}]",28,11,17


In [24]:
#Drop the columns which are not needed
df = df.drop("Cupo", "Sección", "Créditos", "Periodo","Cupos Ofrecidos", "Cupos Disponibles")

In [25]:
df_virt = pl.DataFrame()
df_virt = df.filter(pl.col('Campus').is_in(["NO DEFINIDO",
"VIRTUAL",
"EXTERNO UNIVERSIDAD" ] ))
df_virt.sample(10)

Course Code,Course Name,NRC,Parte de Periodo,Campus,Schedule Table,Cupos Inscritos
str,str,i64,str,str,list[struct[6]],i64
"""MISW-4405""","""ANALITICA DE PROCESOS (CICLO 2…",68076,"""SEGUNDO CICLO - 8 SEMANAS""","""VIRTUAL""","[{""I"",""1730-1850"","".VIRT"",""2024-10-07"",""2024-11-30"","".VIRTUAL""}]",10
"""MEDI-3418""","""REANIMACION BASICA Y AVANZADA""",70058,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""V"",""0700-1150"","".FSFB"",""2024-07-29 2024-08-11 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""J"",""0700-1550"","".FSFB"",""2024-07-29"",""2024-08-11 .FUNDACION SANTAFE DE BOGOTA""}]",5
"""MEDI-3407U""","""PEDIATRÍA URGENCIAS""",69636,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".CCOLSUB"",""2024-07-15 2024-07-28 .CLINICA INFANTIL COLSUBSIDIO""}, {""M"",""V"",""0700-1150"","".CCOLSUB"",""2024-07-15"",""2024-07-28 .CLINICA INFANTIL COLSUBSIDIO""}, {""D"",""0700-1850"","".CCOLSUB"",""2024-07-15"",""2024-07-28"","".CLINICA INFANTIL COLSUBSIDIO""}]",3
"""MEDI-3416U""","""GINECOLOGÍA Y OBSTETRICIA UNID…",70017,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""V"",""0700-1150"","".FSFB"",""2024-07-22 2024-07-28 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""J"",""0700-1550"","".FSFB"",""2024-07-22"",""2024-07-28 .FUNDACION SANTAFE DE BOGOTA""}, {""D"",""0700-1850"","".FSFB"",""2024-07-22"",""2024-07-28"","".FUNDACION SANTAFE DE BOGOTA""}]",3
"""MEDI-3417R""","""MEDICINA LEGAL ROTACIÓN""",70039,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""V"",""0700-1150"","".MEDLEG"",""2024-09-23 2024-10-13 .MEDICINA LEGAL""}, {""M"",""J"",""0700-1550"","".MEDLEG"",""2024-09-23"",""2024-10-13 .MEDICINA LEGAL""}, {""D"",""0700-1850"","".MEDLEG"",""2024-09-23"",""2024-10-13"","".MEDICINA LEGAL""}]",6
"""PSIQ-5604""","""INFANTIL""",68648,"""POSGRADO MEDICINA - 26 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""-"",""."",""2024-08-01"",""2025-01-31"",""."",""No info""}]",4
"""PEDI-5602""","""REUMATOLOGÍA""",51819,"""POSGRADO MEDICINA - 26 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""-"",""."",""2024-08-01"",""2025-01-31"","".Bloque"",""M""}]",6
"""MEDI-3406H""","""MEDICINA INTERNA GASTROENTEROL…",69734,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".FSFB"",""2024-07-29 2024-08-04 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""V"",""0700-1150"","".FSFB"",""2024-07-29"",""2024-08-04 .FUNDACION SANTAFE DE BOGOTA""}, {""D"",""0700-1850"","".FSFB"",""2024-07-29"",""2024-08-04"","".FUNDACION SANTAFE DE BOGOTA""}]",4
"""MEDI-3406H""","""MEDICINA INTERNA GASTROENTEROL…",69744,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""M"",""V"",""0700-1150"","".FSFB"",""2024-10-15"",""2024-10-20 .FUNDACION SANTAFE DE BOGOTA""}, {""I"",""J"",""0700-1550"","".FSFB"",""2024-10-15"",""2024-10-20 .FUNDACION SANTAFE DE BOGOTA""}, {""D"",""0700-1850"","".FSFB"",""2024-10-15"",""2024-10-20"","".FUNDACION SANTAFE DE BOGOTA""}]",4
